In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import csv 
import re
import matplotlib.pyplot as plt
from IPython.display import Image

import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
df = pd.read_csv('Sources/fiuba_6_avisos_detalle_concat.csv', delimiter=',')
descripcion = pd.read_csv('Sources/onehotencoder.csv', delimiter=',')

In [4]:
df.head()

,Unnamed: 0,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [14]:
df2 = df[['nombre_area']]

In [ ]:
df = df[['idaviso']]

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df2['nombre_area'])
df2['nombre_area'] = le.transform(df2['nombre_area'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
df2.head(2)

,nombre_area
0,51
1,90


In [10]:
descripcion[[-1]]= df2[['nombre_area']]

In [12]:
descripcion.drop(['Unnamed: 0'],axis=1,inplace=True)
descripcion.head(5)

,0,00,000,0000,0000hs,0018,0040,0044,0048,00h,...,zulyana,zum,zur,zurich,zusatzlich,zuul,zwischen,zzetto,zzona,-1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,51
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,90
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,86
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,145
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30


In [32]:
df3 = df2.groupby('nombre_area').size()

In [37]:
df3 = df3[df3>50]

In [39]:
df3.count()

79

In [40]:
from sklearn.cluster import KMeans

In [41]:
kmeans = KMeans(n_clusters=80)

In [42]:
kmeans = kmeans.fit(descripcion)

In [43]:
labels = kmeans.predict(descripcion)

In [44]:
centroids = kmeans.cluster_centers_

In [45]:
labels

array([57, 22, 52, ..., 13, 16, 13], dtype=int32)

In [46]:
df['cluster'] = pd.Series(labels)

In [54]:
df=df[['idaviso','cluster']]

In [55]:
df.head()

,idaviso,cluster
0,1111556097,57
1,1111949392,22
2,1112145935,52
3,1112146010,19
4,1112211475,30


In [56]:
df.to_csv('idaviso-cluster.csv')

In [49]:
centroids.shape

(80, 47750)

In [50]:
type(centroids)

numpy.ndarray

In [51]:
df2 = pd.DataFrame(centroids)

In [52]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,47740,47741,47742,47743,47744,47745,47746,47747,47748,47749
0,0.000000e+00,0.000000,0.000000e+00,-2.710505e-20,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,170.200000
1,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,35.333333
2,8.968610e-04,0.000897,5.583641e-18,4.065758e-18,6.979551e-19,6.979551e-19,6.979551e-19,6.979551e-19,6.979551e-19,1.395910e-18,...,1.111307e-18,6.979551e-19,1.395910e-18,2.791821e-18,6.979551e-19,1.395910e-18,1.395910e-18,6.979551e-19,6.979551e-19,117.890583
3,4.336809e-19,0.000000,0.000000e+00,-2.710505e-20,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000
4,1.387779e-17,0.001225,4.119968e-18,1.225490e-03,5.149960e-19,5.149960e-19,5.149960e-19,5.149960e-19,5.149960e-19,1.029992e-18,...,2.168404e-19,5.149960e-19,1.029992e-18,2.059984e-18,5.149960e-19,1.029992e-18,1.029992e-18,5.149960e-19,5.149960e-19,80.506127


In [53]:
df2.to_csv('centroides.csv')